# PyCrystEM Demo @ IUCr 2017

PyCrystEM is an open-source library for crystallographic electron microscopy. Multi-dimensional data processing tools build on the HyperSpy library.

This notebook demonstrates the use of PyCrystEM to: form so called 'virtual diffraction images' by plotting the diffracted intensity in a particular pixel in the reciprocal space images, learn the component patterns making up the data using machine learning techniques, and perform data enhancement as a pre-processing step for further analysis such as indexation and orientation mapping. 

## Authors

08/06/17 Duncan Johnstone - Developed for Trondheim Diffraction Workshop

# Requirements

Pycrystem 0.1

HyperSpy 1.3

PyMatGen

## Contents

1. <a href='#loa'> Loading & Inspection</a>
2. <a href='#vdf'> Virtual Diffraction Imaging</a>
3. <a href='#ml'> Machine Learning SPED Data</a>
4. <a href='#pre'> Pre-processing & Peak Finding</a>
5. <a href='ori'> Orientation Mapping by Template Matching</a>

Import pycrystem

In [1]:
%matplotlib tk
import pycrystem as pc
import pymatgen as pmg
import numpy as np
from scipy.constants import pi

C:\Users\Duncan\AppData\Local\conda\conda\envs\pycrystem\lib\site-packages\pymatgen\util\plotting_utils.py:3: UserWarning: pymatgen.util.plotting_utils has been renamed pymatgen.util.plotting. This stub will be removed in PMG 5.0.
  warnings.warn("pymatgen.util.plotting_utils has been renamed "


## <a id='loa'></a> 1. Loading and Inspection

Load the SPED data acquired from the nanowire

In [2]:
dp = pc.load('nanowire_precession.hdf5')

Look at what kind of object 'dp' is

In [3]:
dp

<ElectronDiffraction, title: , dimensions: (30, 100|144, 144)>

Set important experimental parameters using the built in function

In [4]:
dp.set_experimental_parameters(accelerating_voltage=300.,
                               camera_length=40.,
                               scan_rotation=270.,
                               convergence_angle=2.,
                               exposure_time=10.)
# Setting a non-standard item
dp.metadata.set_item("General.title", 'GaAs Nanowire')

Inspect the metadata

In [5]:
dp.metadata

├── Acquisition_instrument
│   └── TEM
│       ├── Detector
│       │   └── Diffraction
│       │       ├── camera_length = 40.0
│       │       └── exposure_time = 10.0
│       ├── accelerating_voltage = 300.0
│       ├── beam_energy = 300.0
│       ├── camera_length = 0.21000000000000002
│       ├── convergence_angle = 2.0
│       └── scan_rotation = 270.0
├── General
│   ├── original_filename = nanowire_precession.blo
│   ├── time = (2014, 12, 8)
│   └── title = GaAs Nanowire
└── Signal
    ├── binned = False
    ├── signal_origin = 
    └── signal_type = electron_diffraction

Set the diffraction pattern calibration

In [6]:
dp.set_calibration(0.0035)

Plot the data to inspect it

In [7]:
dp.plot(vmin=0, vmax=100)

## <a id='vdf'></a> 2. Virtual Diffraction Imaging

Plot an interactive virtual image integrating intensity within a circular subset of pixels in the diffraction pattern

In [8]:
roi = pc.roi.CircleROI(cx=0.,cy=0, r_inner=0, r=0.02)
dp.plot_interactive_virtual_image(roi=roi)

Get the virtual diffraction image associated with the last integration window used interactively

In [9]:
vdf=dp.get_virtual_image(roi)
vdf=vdf.as_signal2D((0,1))
vdf.change_dtype('float32')
vdf.save('vdfeg.tif')

Plot the data with an adjustable marker indicating where to crop the scan region

In [10]:
reg = pc.roi.RectangularROI(left=50, top=837.5, right=290, bottom=1237.5)
dp.plot(vmin=0, vmax=100)
reg.add_widget(dp)

Crop the dataset based on the region defined above

In [11]:
dpc = reg(dp)

Plot the cropped dataset

In [12]:
dpc.plot(vmin=0, vmax=100)

# <a id='ml'></a> 3. Unsupervised learning of component patterns

Perform singular value decomposition (SVD) of the data

In [13]:
dpc.change_dtype('float')
dpc.decomposition(True, algorithm='svd')

Obtain a "Scree plot" by plotting the fraction of variance described by each principal component

In [14]:
dpc.plot_explained_variance_ratio()

Plot the decomposition results and have a look at them

In [15]:
dpc.plot_decomposition_results()

Perform non-negative matrix factorisation (NMF)

In [16]:
dpc.decomposition(True, algorithm='nmf', output_dimension=3)

Plot the NMF results

In [17]:
dpc.plot_decomposition_results()

# <a id='pre'></a> 4. Pre-processing & Peak Finding

In [18]:
dp = pc.load('nanowire_precession.hdf5')

Perform a background subtraction by radially integrating the 2D diffraction pattern and fitting a model containing a lorentzian, exponential and linear function

In [19]:
dpb = dp.remove_background(method='model', saturation_radius=1)
dpb.plot(vmin=0, vmax=100)

A Jupyter Widget

A Jupyter Widget

3000/|/100%|| 3000/3000 [00:30<00:00, 278.65it/s]                                                                      

Obtain a radially integrated diffraction profile and plot it

In [20]:
rp = dp.get_radial_profile()
rp.plot()

A Jupyter Widget

Interactively tune peak finding parameters

In [21]:
dp.find_peaks_interactive()

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

Perform peak finding on all diffraction patterns in data

In [22]:
peaks = dp.find_peaks()

A Jupyter Widget

A Jupyter Widget

Check the peaks object

In [23]:
peaks

<DiffractionVectors, title: , dimensions: (30, 100|)>

# <a id='ori'></a> 5. Orientation Mapping & Template Matching

Load test data for orientation & phase matching

In [24]:
test_data = pc.load('test_data.hspy')

Define structures against which to index the data

In [25]:
graphite = pmg.Structure.from_file('./graphite.cif')

In [26]:
si = pmg.Element("Si")
lattice = pmg.Lattice.cubic(5.431)
silicon = pmg.Structure.from_spacegroup("Fd-3m",lattice, [si], [[0, 0, 0]])

Define orientations to include in the template library

In [27]:
rot_list = []
nstep=119
for i in np.arange(nstep):
    theta = (i*59.5/(nstep-1))/180*pi
    rot_list.append((theta, 0., 0.))

Simulate template library

In [28]:
edc = pc.ElectronDiffractionCalculator(50, 5e-2)
diff_gen = pc.DiffractionLibraryGenerator(edc)
struc_lib = dict()
struc_lib["gr"] = (graphite, rot_list)
struc_lib['si'] = (silicon, rot_list)
library = diff_gen.get_diffraction_library(struc_lib,
                                            calibration=1.2/128,
                                            reciprocal_radius=1.,
                                            representation='euler')

Perform indexation by template matching

In [30]:
from pycrystem.indexation_generator import IndexationGenerator
indexer = IndexationGenerator(test_data, library)
match_results = indexer.correlate()

A Jupyter Widget


270/|/100%|| 270/270 [00:22<00:00, 32.23it/s]                                                                          

Transform matching results to crystallographic map

In [31]:
cryst_map = match_results.get_crystallographic_map()

A Jupyter Widget

Plot phase map

In [33]:
test_data.plot()

In [32]:
cryst_map.get_phase_map().plot()

Plot orientation map

In [34]:
cryst_map.get_orientation_image().plot()

A Jupyter Widget